<h1><center> Dashboarding en Python - Gradio y Dash </center></h1>

<h3><center>  Autor: Cristian Quintana, mail: cquintan@cern.ch </center></h3>



Tenemos dos alternativas para la gestión de los paquetes que vamos a necesitar. Ambas estan relacionadas con el papel
que vamos a desempeñar: ¿Seremos los encargados de desarrollar la app? ¿Seremos meros usuarios o testers?


## Generar un nuevo entorno virtual:
Si nuestro papel es desarrollar la app tendremos que ser nosotros los encargados de instalar todos los paquetes que
consideremos oportunos. Para ello es recomendable crear un nuevo entorno virtual. De esta forma aislaremos el entorno
de trabajo de la app del resto del equipo (más o menos). 

    Procedemos a abrir una terminal (LINUX)
	Crear directorio y hacer cd:
 
mkdir Dash_Jupyter

cd Dash_Jupyter


	Generar un entorno virtual de python y activarlo:
python3 -m venv DashJ

source DashJ/bin/activate

	Instalar paquetes:
python -m pip install dash==2.8.1 pandas==1.5.3 jupyter jupyterlab jupyter_dash==0.4.2 dash_bootstrap_components openpyxl datetime kaleido gradio

    Abrir jupyter lab:
jupyter lab

## Descargar desde un repositorio:
La otra posibilidad es que la app exista y queramos lanzarla. Si el desarrollador hizo lo propio, ya dispondremos de este entorno
virtual, y nuestra labor será cargarlo.
    Primero hay que tener instalado git:
    
sudo apt install git

    Copiar (clonar) el repositorio
    
cd ~              #cd a la carpeta home, o a otra dónde se quiera descargar la app

git clone https://github.com/CQuintanaSE/Dash_app.git

## Instalar directamente con pip:




In [1]:
!pip install gradio dash numpy pandas jupyter_dash dash_bootstrap_components openpyxl datetime kaleido
print("DONE!")

  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)

[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
DONE!


## Importar, importar e importar

In [2]:
# IMPORT Packages
from dash import Dash, dcc, html, Input, Output, State # Dash
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash   # Dash for Jupyter
import subprocess                      # Download files
import os
import pandas as pd                    # data
import plotly.express as px            # plot easy
import plotly.graph_objects as go      # plot hard
from datetime import date
import gradio as gr                    # gradio dashboard
import numpy as np
import plotly.io as pio
pio.renderers.default = 'notebook'
print("DONE!")

DONE!


# EJERCICIO con GRADIO: Analizando una imagen 

En este primer ejemplo utilizaremos gradio para analizar una imagen que tengamos descargada en nuestro equipo.

Para ello crearemos una función que sea capaz de contar el número de píxeles de una imagen los cuales tengan un valor por debajo de un umbral.
Con Gradio generaremos la interfaz capaz de seleccionar y leer la imagen, pasando dicha imagen a la función de conteo y mostrar el resultado en otro campo.

In [3]:
""" CountPixels()
    Toma como input una imagen (de Gradio), y usando métodos del paquete numpy retorna el número de píxeles (str) con un valor
    por debajo de un valor umbral (<=10).
"""
def CountPixels(input_img):
   # import cv2
   # from PIL import Image
   # img = Image.open(input_img);
    dark_count = np.sum(np.array(input_img) <= 10)
    return  f"El número de pixeles oscuros es {dark_count}"

# Generamos la app
demo = gr.Interface(CountPixels, gr.Image(), "text")  # Interfaz: función, input, output
demo.launch(share = True)  # Lanzoms la app | share = True --> Genera url pública

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e0ea029fd94ecc4301.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Trabajo para el estudiante:

Repite esta aplicación pero:

    - Cambia la función para que retorne las estadísticas más importantes
    - Muéstralas con Gradio

In [4]:
""" statistics()
    Toma como input una imagen (de Gradio), y usando métodos del paquete numpy retorna las estadísticas más importantes
    en cuanto al color en ByN de los píxeles.
"""
def statistics(input_img):
    # Imagen a numpy array
    pixels = np.array(input_img)
    # Pasar a imagen en blanco y negro (media en el tercer eje)
    pixels = np.mean(pixels, axis=2)
    # Eleborar string de texto con formato incluyendo los datos
    text = "Estadísticas: media={:.2f} | mediana={:.2f} | std={:.2f} | max={:.0f} | min={:.0f}".format(
        np.mean(pixels), np.median(pixels), np.std(pixels), np.max(pixels), np.min(pixels),
    )
    return  text

# Generamos la app
demo = gr.Interface(statistics, gr.Image(), "text")  # Interfaz: función, input, output
demo.launch(share = False)  # Lanzoms la app | share = True --> Genera url pública

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# EJERCICIO con DASH (1): Leer de una Google spreadsheet (excel)

Primer ejercicio de desarrollo de una app con Dash. En este ejemplo descargaremos y leeremos una Google Spreadsheet (también se explicarán los pasos para hacer el documento acesible),
usaremos pandas para generar un dataframe y haremos un tratamiento estadístico de los datos usando Plotly. Por último generaremos el dashboard con las imagenes previas.


ENLACE: https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/edit?usp=sharing

## Descargar y dar formato
Pasos previos:

------> Excel

Hacer el documento público

Copiar el enlace 

Generar un link de descarga: https://www.graytechnical.com/blog/google-drive-online-download-link-generator-files-sheets-and-docs-online/


In [5]:
""" download_as() 
Función para la descarga de un fichero de datos desde la web, renombra si se le
pasa un str name.
Inputs:
    url (str): url de la página web 
    name (str): nombre para renombrar el fichero (por defecto no renombra)
    dir (str): directorio de descarga ("Data_files/" por defecto)
"""
def download_as(url, name="", dir="Data_files/"):
    # Rename 
    if name != "":
        command = f'wget --output-document={dir+name} {url}'
    else:
        command = f'wget {url} -P {dir}'
        
    subprocess.run(command, shell=True, stderr=subprocess.DEVNULL)
    print("Done! -> Downloaded file")


# DOWNLOAD
# Descarga en la carpeta por defecto
filename = "Dashboard_excel.xlsx" # obligatorio renombrar con google spreadsheets
url = "https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/export?format=xlsx"
download_as(url, filename) # Descarga con renombrado

# FORMATEAR a dataframe
excel_data = pd.read_excel("Data_files/"+filename, skiprows = 5)
excel_data.head()

Done! -> Downloaded file


,Nombre,Valoracion Curso,Valoracion Dash,Conjunto,Genero
0,Alba,1,1,1,M
1,Pedro,3,1,2,M
2,Alicia,3,3,1,N
3,Juan,3,3,1,M
4,Carlos,1,1,3,H


## Contruyendo la app
Seguiremos el esquema de la presentación:

    Imports (ya hecho) | Parámetros | Funciones | App | Cierre

In [6]:
#--------------- Parámetros y Objetos ----------------------------------------------------------------------------
# Definimos los parámetros y objetos: dimensiones, títulos, figuras, imágenes, etc...

# Título principal y su estilo
header = """ \t Encuesta sobre el curso
         """
style_header = {'margin-left':'500px', 'margin-top':'40px', 'margin-right':'15px', 'margin-bottom':'40px'}

# Figuras del dashboard (Nota!: no hace falta crearlas completamente ahora)
Histo1 = px.histogram(excel_data, x="Valoracion Curso", nbins=5, range_x=[0.5, 5.5])
Histo2 = px.histogram(excel_data, x="Valoracion Dash", nbins=5, range_x=[0.5, 5.5])
Histo3 = px.histogram(excel_data, x="Conjunto",  nbins=5, range_x=[0.5, 5.5])
# go.FigureWidget(Histo1)  # Mostrar en el notebook

#--------------- Funciones ---------------------------------------------------------------------------------------
""" replot()
Toma los 3 histogramas (plotly) de la app y los acualiza en base a los nuevos datos data1, data2, data3 (pandas df)
"""
def replot(Histo1, Histo2, Histo3, data1, data2, data3):
    Histo1.update_traces(x=data1)
    Histo2.update_traces(x=data2)
    Histo3.update_traces(x=data3)
#--------------- APP ---------------------------------------------------------------------------------------------

# Instanciamos -> objeto "app"
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG]) # usamos un tema predefinido "CYBORG"

# Layout de la app
# Distribución por: contenedor{ filas y columnas }
app.layout = dbc.Container(
    [
        dbc.Row(html.H5(header), style = style_header),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(figure = Histo1, id="H1")),
                dbc.Col(dcc.Graph(figure = Histo2, id="H2")),
                dbc.Col(dcc.Graph(figure = Histo3, id="H3")),
            ]
        ),
         dbc.Row([dbc.Col(html.Button('Actualizar datos', id='update-data', n_clicks=0), width = 2),
                  dbc.Col(html.Div(children="Text", id='update-date'), width = 8),
                 ]),
    ]
)

# Gestión de los eventos
""" Cabecera (Decorador) "callback" y función de actualización de la página
Cabecera:
    Inputs:  identificador, parámetro a importar
    Outputs: identificador, parámetro a exportar

Función update_output()
    Input: n_clicks, lo toma del input
    Return: str, e histogramas

ORDEN!!! -> Los inputs y outputs deben de guardar el orden de la signature de la función y del return
"""
@app.callback(
    Output('update-date', 'children'),
    Output('H1', 'figure'),
    Output('H2', 'figure'),
    Output('H3', 'figure'),
    Input('update-data', 'n_clicks'),
)
def update_output(n_clicks):
    if n_clicks>0: 
        filename = "Dashboard_excel.xlsx"
        url = "https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/export?format=xlsx"
        download_as(url, filename)
        excel_data = pd.read_excel("Data_files/"+filename, skiprows = 5)
        data1, data2, data3 = excel_data["Valoracion Curso"], excel_data["Valoracion Dash"], excel_data["Conjunto"]
        replot(Histo1, Histo2, Histo3, data1, data2, data3)
        text = ("Actualizado! ->  "+subprocess.run("date", shell=True, capture_output=True, text=True).stdout.strip())
    else:
        text = ("Los ficheros de datos no han sido actualizados desde el inicio de la sesión")

    return text, Histo1, Histo2, Histo3

#--------------- Cierre  ----------------------------------------------------------------------------------------
# Se lanza la app
# debug=True -> modo debug
if __name__ == "__main__":
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/


## Trabajo para el estudiante:

Repite esta aplicación pero:

    - Añade una cuarta columna "Genero" al spreadsheet (Rellenar H M N) 
    
    - Añade esa cuarta columna a la lectura para que se incluya en el df
    
    - Genera una cuarta figura con plotly con la nueva información (diferente a un histograma, pie?)
    
    - Copia la estructura de la app pero muestra las gráficas en formato 2X2 en vez de 3X1 (modificar el layout)

In [7]:
# DESCARGA Y FORMATO del DataFrame

# Descarga en la carpeta por defecto
filename = "Dashboard_excel.xlsx" # obligatorio renombrar con google spreadsheets
url = "https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/export?format=xlsx"
download_as(url, filename) # Descarga con renombrado

# FORMATEAR a dataframe
excel_data = pd.read_excel("Data_files/"+filename, skiprows = 5)
excel_data.head()

Done! -> Downloaded file


,Nombre,Valoracion Curso,Valoracion Dash,Conjunto,Genero
0,Alba,1,1,1,M
1,Pedro,3,1,2,M
2,Alicia,3,3,1,N
3,Juan,3,3,1,M
4,Carlos,1,1,3,H


In [8]:
# Crear (y probar la gráfica)

Graph1 = px.pie(excel_data, values='Conjunto', names='Genero', title='Titulo')
go.FigureWidget(Graph1)

FigureWidget({
    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': 'Genero=%{label}<br>Conjunto=%{value}<extra></extra>',
              'labels': array(['M', 'M', 'N', 'M', 'H', 'M'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'type': 'pie',
              'uid': '51a09f3b-3e07-43c4-bfed-1409a6e6415e',
              'values': array([1, 2, 1, 1, 3, 4])}],
    'layout': {'legend': {'tracegroupgap': 0}, 'template': '...', 'title': {'text': 'Titulo'}}
})

In [9]:
#--------------- Parámetros y Objetos ----------------------------------------------------------------------------
# Definimos los parámetros y objetos: dimensiones, títulos, figuras, imágenes, etc...

# Título principal y su estilo
header = """ \t Encuesta sobre el curso
         """
style_header = {'margin-left':'500px', 'margin-top':'40px', 'margin-right':'15px', 'margin-bottom':'40px'}

# Figuras del dashboard (Nota!: no hace falta crearlas completamente ahora)
Histo1 = px.histogram(excel_data, x="Valoracion Curso", nbins=5, range_x=[0.5, 5.5])
Histo2 = px.histogram(excel_data, x="Valoracion Dash", nbins=5, range_x=[0.5, 5.5])
Histo3 = px.histogram(excel_data, x="Conjunto",  nbins=5, range_x=[0.5, 5.5])
Graph1 = px.pie(excel_data, values='Conjunto', names='Genero', title='Gráfica tarta')

# go.FigureWidget(Histo1)  # Mostrar en el notebook

#--------------- Funciones ---------------------------------------------------------------------------------------
""" replot()
Toma los 3 histogramas (plotly) de la app y los acualiza en base a los nuevos datos data1, data2, data3 (pandas df)
"""
def replot(Histo1, Histo2, Histo3, Graph1, data1, data2, data3):
    Histo1.update_traces(x=data1)
    Histo2.update_traces(x=data2)
    Histo3.update_traces(x=data3)
    Graph1.update_traces(values=data3)

#--------------- APP ---------------------------------------------------------------------------------------------

# Instanciamos -> objeto "app"
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG]) # usamos un tema predefinido "CYBORG"

# Layout de la app
# Distribución por: contenedor{ filas y columnas }
app.layout = dbc.Container(
    [
        dbc.Row(html.H5(header), style = style_header),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(figure = Histo1, id="H1")),
                dbc.Col(dcc.Graph(figure = Histo2, id="H2")),
            ]
        ),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(figure = Histo3, id="H3")),
                dbc.Col(dcc.Graph(figure = Graph1, id="G1")),
            ]
        ),
         dbc.Row([dbc.Col(html.Button('Actualizar datos', id='update-data', n_clicks=0), width = 2),
                  dbc.Col(html.Div(children="Text", id='update-date'), width = 8),
                 ]),
    ]
)

# Gestión de los eventos
""" Cabecera (Decorador) "callback" y función de actualización de la página
Cabecera:
    Inputs:  identificador, parámetro a importar
    Outputs: identificador, parámetro a exportar

Función update_output()
    Input: n_clicks, lo toma del input
    Return: str, e histogramas

ORDEN!!! -> Los inputs y outputs deben de guardar el orden de la signature de la función y del return
"""
@app.callback(
    Output('update-date', 'children'),
    Output('H1', 'figure'),
    Output('H2', 'figure'),
    Output('H3', 'figure'),
    Output('G1', 'figure'),
    Input('update-data', 'n_clicks'),
)
def update_output(n_clicks):
    if n_clicks>0: 
        filename = "Dashboard_excel.xlsx"
        url = "https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/export?format=xlsx"
        download_as(url, filename)
        excel_data = pd.read_excel("Data_files/"+filename, skiprows = 5)
        data1, data2, data3 = excel_data["Valoracion Curso"], excel_data["Valoracion Dash"], excel_data["Conjunto"]
        replot(Histo1, Histo2, Histo3, Graph1, data1, data2, data3)
        text = ("Actualizado! ->  "+subprocess.run("date", shell=True, capture_output=True, text=True).stdout.strip())
    else:
        text = ("Los ficheros de datos no han sido actualizados desde el inicio de la sesión")

    return text, Histo1, Histo2, Histo3, Graph1

#--------------- Cierre  ----------------------------------------------------------------------------------------
# Se lanza la app
# debug=True -> modo debug
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


# EJERCICIO con DASH (2): Leer desde "la web" (Covid Data)

En este segundo ejercicio con Dash haremos acopio de todos los conocimientos adquiridos en el curso. Intentaremos generar una app que hacceda a la información de una web ajena a nosotros, haga un tratamiento de los datos y los puestre en nuestro dashboard.

In [10]:
# DOWNLOAD datafiles
# Ejemplo de descarga sin la función
url = 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/csv/data.csv'
download_as(url, "covid_Europe.csv")

Done! -> Downloaded file


In [11]:
# Cargar los datos
data = (pd.read_csv("Data_files/covid_Europe.csv"))
print(data.head())

      dateRep  day  month  year    cases  deaths countriesAndTerritories  \
0  23/10/2022   23     10  2022   3557.0     0.0                 Austria   
1  22/10/2022   22     10  2022   5494.0     4.0                 Austria   
2  21/10/2022   21     10  2022   7776.0     4.0                 Austria   
3  20/10/2022   20     10  2022   8221.0     6.0                 Austria   
4  19/10/2022   19     10  2022  10007.0     8.0                 Austria   

  geoId countryterritoryCode  popData2020 continentExp  
0    AT                  AUT      8901064       Europe  
1    AT                  AUT      8901064       Europe  
2    AT                  AUT      8901064       Europe  
3    AT                  AUT      8901064       Europe  
4    AT                  AUT      8901064       Europe  


In [12]:
# Formato de los datos -> Columna de fechas
data["Date"] = pd.to_datetime(data["dateRep"], format="%d/%m/%Y")
# Generamos un subset con los datos de españa
spain_data = data[data["countriesAndTerritories"] == "Spain"]
print(spain_data.head())

          dateRep  day  month  year    cases  deaths countriesAndTerritories  \
26816  21/10/2022   21     10  2022  25422.0    75.0                   Spain   
26817  14/10/2022   14     10  2022  20652.0    68.0                   Spain   
26818  07/10/2022    7     10  2022  10843.0    68.0                   Spain   
26819  04/10/2022    4     10  2022   8114.0    83.0                   Spain   
26820  30/09/2022   30      9  2022  10721.0    95.0                   Spain   

      geoId countryterritoryCode  popData2020 continentExp       Date  
26816    ES                  ESP     47332614       Europe 2022-10-21  
26817    ES                  ESP     47332614       Europe 2022-10-14  
26818    ES                  ESP     47332614       Europe 2022-10-07  
26819    ES                  ESP     47332614       Europe 2022-10-04  
26820    ES                  ESP     47332614       Europe 2022-09-30  


In [13]:
# Generamos una lista con los países incluidos en el dataframe
regions = data["countriesAndTerritories"].sort_values().unique()
print(regions)

['Austria' 'Belgium' 'Bulgaria' 'Croatia' 'Cyprus' 'Czechia' 'Denmark'
 'Estonia' 'Finland' 'France' 'Germany' 'Greece' 'Hungary' 'Iceland'
 'Ireland' 'Italy' 'Latvia' 'Liechtenstein' 'Lithuania' 'Luxembourg'
 'Malta' 'Netherlands' 'Norway' 'Poland' 'Portugal' 'Romania' 'Slovakia'
 'Slovenia' 'Spain' 'Sweden']


## Contruimos la app

Hay que tener en cuenta que no toda la información que parte de los parámetros que vamos a necesitar están incluidos en las celdas previas

In [14]:
#--------------- Parametros y Objetos ----------------------------------------------------------------------------
# FIGURES
# WH_set1 = [900,400]

figs = {}
figs["CasesAndDeaths"] = go.Figure() # Sólo instanciamos
figs["CasesCountries"] = go.Figure()

#--------------- Funciones -----------------------------------------------------------------------------

""" Crear una función que me permita descargar el set de datos de una url en un directorio determinado
    Si se le aporta como argumento un nombre, el fichero descargado es renombrado
    Opcional: Return o print mensaje "Compleatado"
""" 
def download_as(url, name="", dir="Data_files/"):
    # Rename 
    if name != "":
        command = f'wget --output-document={dir+name} {url}'
    else:
        command = f'wget {url} -P {dir}'
        
    subprocess.run(command, shell=True, stderr=subprocess.DEVNULL)
    print("Done! -> Downloaded file")
    

""" La función lee el fichero de datos con pandas.
    Como primer argumento un str con el nombre de un país -> sub dataset del país
    Como segundo argumento lista de str de paises -> lista de sub datasets
    retorna: datos completos, datos del país, datos de los paises  (lista)
"""
def get_datasets(Country = "Spain", Countries = ["Spain","France","Germany"]):
    # Leer csv
    data = (pd.read_csv("Data_files/covid_Europe.csv"))
    # Crear columna "Date" con formato datetime
    data["Date"] = pd.to_datetime(data["dateRep"], format="%d/%m/%Y")
    # Generar dataset y lista de datasets
    country_data = data[data["countriesAndTerritories"] == Country]
    countries_data = [data[data["countriesAndTerritories"] == _c] for _c in Countries]

    return data, country_data, countries_data

""" Función que genera las figuras con PLOTLY!
    Tiene como argumentos los mismos argumentos que la anterior además de el return de la anterior
    country (str), country_data (dataset), countries [str], countries_data [datasets]
"""
def gen_figs(country, country_data, countries, countries_data):
    # https://plotly.com/python/line-charts/#line-plot-modes
    
    figs = {}
    # Figures of one single country (First section)
    # Figure CasesAndDeaths
    figs["CasesAndDeaths"] = go.Figure()
    figs["CasesAndDeaths"].add_trace(go.Scatter(x=country_data["Date"], y=country_data["cases"]/country_data["cases"].max(),
                        mode='lines',
                        name='Cases'))
    figs["CasesAndDeaths"].add_trace(go.Scatter(x=country_data["Date"], y=country_data["deaths"]/country_data["deaths"].max(),
                        mode='lines',
                        name='Deaths'))
    figs["CasesAndDeaths"].update_layout(
                       xaxis_title='Date',
                       yaxis_title='Normallized to max',
                       title='Visualizando '+country)

    # Figures of one multiple countries (Second section)
    # Figure CasesCountries
    figs["CasesCountries"] = go.Figure()
    N_countries = len(countries)
    for i in range(N_countries):
        tdata = countries_data[i]
        figs["CasesCountries"].add_trace(go.Scatter(x=tdata["Date"], y=tdata["cases"],
                    mode='lines',
                    name=countries[i]))

    return figs

#--------------- Contruyendo la app ------------------------------------------------------------------------------

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])

header = """Dashboard interactivo con los últimos datos de Covid para Europa hasta final del seguimiento. 
         En los diferentes apartados podrás acceder a los datos individuales de cada país europeo así como realizar comparaciones entre ellos
         o seleccionar el rango de fechas a estudiar.
         """

style_1 = {'margin-left':'15px', 'margin-top':'10px', 'margin-right':'15px', 'margin-bottom':'15px'}

app.layout = html.Div(
    [
        dbc.Row(dbc.Col(html.Img(src="assets/LogoDash.png"), width = 10)),
        dbc.Row(dbc.Col(html.H5(header), width = 10), style = style_1),
        dbc.Row([dbc.Col(html.Button('Actualizar datos', id='update-data', n_clicks=0),
                         style = style_1, width = 1),
                 dbc.Col(html.Div(children="", id='update-date'),
                         style = style_1, width = 9),
                 
                ]),
        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

        dbc.Row(
            [
                dbc.Col([dcc.Dropdown(
                            id="region-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in regions
                            ],
                            value="Spain",
                            clearable=False,
                            className="dropdown",
                        ), 
                        dcc.DatePickerRange(
                        id='datePicker',
                        min_date_allowed=date(2020, 1, 1),
                        max_date_allowed=date(2022, 10, 19),
                        start_date=date(2020, 1, 1),
                        end_date=date(2022, 10, 19),
                        ),
                        ],
                        width = 3,style = style_1),
                dbc.Col(dcc.Graph(figure = figs["CasesAndDeaths"], id="F_CD"), width = 7)
            ]
        ),
        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

        dbc.Row(
            [
                dbc.Col(dcc.Dropdown(
                            id="countries-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in regions
                            ],
                            value=["Spain", "Portugal", "Germany"],
                            multi=True,
                            clearable=False,
                            className="dropdown",
                        ), 
                        width = 3,style = style_1),
                dbc.Col(dcc.Graph(figure = figs["CasesCountries"], id="F_CC"), width = 7)
            ]
        ),

        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

    ]
)

@app.callback(
    Output('update-date', 'children'),
    Output('update-data', 'n_clicks'),
    Output('F_CD', 'figure'),
    Output('F_CC', 'figure'),
    Input('update-data', 'n_clicks'),
    Input('region-filter', 'value'),
    Input('countries-filter', 'value'),
    Input('datePicker', 'start_date'),
    Input('datePicker', 'end_date')
)
def update_output(n_clicks, region, countries, start_date, end_date):
    if not isinstance(countries, list):
        countries = [countries]
    
    if n_clicks > 0: 
        url = 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/csv/data.csv'
        download_as(url, name = "covid_Europe.csv")
        text = "Actualizado! -> "+subprocess.run("date", shell=True, capture_output=True, text=True).stdout.strip()
    else:
        text = "Los ficheros de datos no han sido actualizados desde el inicio de la sesión"


    data, country_data, countries_data = get_datasets(Country = region, Countries = countries)

    if start_date != None and end_date != None:
        country_data = country_data[(country_data['Date'] >= start_date) & (country_data['Date'] <= end_date)]
    
    figs = gen_figs(region, country_data, countries, countries_data)
    #return text, 0, figs["CasesAndDeaths"], figs["CasesCountries"]
    return text, 0, *figs.values()


#--------------- Launch/Update the app ------------------------------------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


# Biblio

https://dash.plotly.com/

https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/

